In [46]:
import cv2
import imutils
from timeit import default_timer as timer

In [47]:
def test_classifier(img_path, img_path_2, classifier_model_path):
    haar_cascade = cv2.CascadeClassifier(classifier_model_path)
    assert(not haar_cascade.empty())
    img = cv2.imread(img_path)
    start = timer()
    faces_rect = haar_cascade.detectMultiScale(img, 1.1, 9)
    end = timer()
    diff = end - start 
    print(diff)
    for (x, y, w, h) in faces_rect: 
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    cv2.imwrite(img_path_2, img)

In [48]:
test_classifier('d:/dane/face_recognition/faces.jpg', 'd:/dane/face_recognition/faces2.jpg', 'd:/Dane/face_recognition/haarcascade_frontalface_default.xml')

0.19555529998615384


In [49]:
test_classifier('d:/dane/face_recognition/faces.jpg', 'd:/dane/face_recognition/faces3.jpg', 'd:/Dane/face_recognition/haarcascade_frontalface_alt.xml')

0.25077090004924685


In [71]:
def test_dnn_detector(model_file, config_file, use_gpu, src_img_path, target_img_path, threshold):
    img = cv2.imread(src_img_path)
    model = cv2.dnn.readNetFromCaffe(config_file, model_file)
    if use_gpu:
        model.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)
    else:
        model.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        model.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)  
        
    img_height = img.shape[0]
    img_width = img.shape[1]        
    
    start = timer()
    
    blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), [104, 117, 123], False, False)
    model.setInput(blob)
    detections = model.forward()
    
    end = timer()
    diff = end - start 
    print(diff)
    
    img_cpy = img.copy()

    boxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > threshold:
            x1 = int(detections[0, 0, i, 3] * img_width)
            y1 = int(detections[0, 0, i, 4] * img_height)
            x2 = int(detections[0, 0, i, 5] * img_width)
            y2 = int(detections[0, 0, i, 6] * img_height)
            boxes.append([x1, y1, x2, y2])
            cv2.rectangle(
                img_cpy,
                (x1, y1),
                (x2, y2),
                (0, 255, 0),
                int(round(img_height / 150)),
                8,
            )
        
    cv2.imwrite(target_img_path, img_cpy)    

In [72]:
model_file = "d:/Dane/face_recognition/res10_300x300_ssd_iter_140000.caffemodel"
config_file = "d:/Dane/face_recognition\deploy.prototxt"
src_img_path = "d:/dane/face_recognition/faces.jpg"
target_img_path = 'd:/dane/face_recognition/faces4.jpg'

In [77]:
test_dnn_detector(model_file, config_file, True, src_img_path, target_img_path, 0.3)

0.03905309992842376
